In [5]:
import cv2
import torch
import numpy as np
import torch
import torch.nn as nn

# Define the model architecture
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(64 * 16 * 16, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 1)
)

# Load the pre-trained model weights
model.load_state_dict(torch.load('Cycle_model.pth'))
model.eval()

def processframe(frame):
    # Resize frame to the expected input size of the model
    # The model expects an input of size (64, 16, 16) after the conv and pool layers
    # You need to adjust this to match the input size of your model before the first conv layer
    resized_frame = cv2.resize(frame, (64, 64))  # Example, resize to 64x64

    # Convert color to RGB (if your model was trained on RGB images)
    rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

    # Normalize the frame
    # Assuming your model was trained with images having pixel values [0,1] or [-1,1]
    # Adjust these values based on how your model was trained
    normalized_frame = rgb_frame / 255.0

    return normalized_frame


# Initialize the webcam
cap = cv2.VideoCapture(0)
...
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Pre-process the frame
    processed_frame = processframe(frame)

    # Convert frame to tensor
    frame_tensor = torch.from_numpy(processed_frame).permute(2, 0, 1).unsqueeze(0).float()

    # Detect bikes
    with torch.no_grad():
        output = model(frame_tensor)
        probability = torch.sigmoid(output).item()

    # Threshold for detection
    threshold = 0.5  # You may adjust this threshold as needed

    # Check if the model detects a bike
    if probability > threshold:
        # Indicate bike detection on the frame
        cv2.putText(frame, "Bike Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Bike Detection', frame)

    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()

